In [107]:
from polygon import RESTClient
import getpass
from datetime import date
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
 
# Returns the current local date
today = date.today()
dateOfMonth = today.day
monthNumber = today.month
yearNumber = today.year

import ftplib
import io
import pandas as pd
import pandas_ta as ta
import requests
import requests_html
import numpy as np

# from pycaret.classification import *
# from pycaret.regression import *

import yfinance as yf
from yahoo_fin.stock_info import get_data, get_top_crypto, get_analysts_info
import yahoo_fin.stock_info as si
import yahoo_fin.options as ops
from yahoo_fin.stock_info import *

# Dow: tickers_dow()
# Nasdaq: tickers_nasdaq()
# S&P500: tickers_sp500()
# Others: tickers_other()

# import tensorflow as tf
import altair as alt

# from benzinga import financial_data
# from benzinga import news_data

import mplfinance as mpf

import requests_cache

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D 

import metricFxns
from metricFxns import *
import vizFxns
from vizFxns import *

from sklearn import metrics
from sklearn import datasets

from yahooquery import Ticker

import ssl
import certifi
import urllib

from yahoo_fin import stock_info as si
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import ExponentialSmoothing
import warnings
import matplotlib.pyplot as plt
import statsmodels.api as sm
# Augmented Dickey-Fuller Test
from statsmodels.tsa.stattools import adfuller

In [108]:
currentDate = str(monthNumber) + "/" + str(dateOfMonth) + "/" + str(yearNumber)
yearPrior = str(monthNumber) + "/" + str(dateOfMonth+1) + "/" + str(yearNumber-1)

In [109]:
roth = pd.read_csv("D:\Roth.csv")
indv = pd.read_csv("D:\Indv.csv")

roth.columns = list(roth.iloc[1].values)
roth = roth.iloc[2:]
roth = roth.iloc[:-1]

indv.columns = list(indv.iloc[1].values)
indv = indv.iloc[2:]
indv = indv.iloc[:-1]

indv = indv[['Symbol','% Of Account','Security Type']]
roth = roth[['Symbol','% Of Account','Security Type']]

current_portfolio = list(indv['Symbol'].values)

In [133]:
historyDfs = []
for ticker in current_portfolio[:10]: 
    try: 
        yahooTickerData = yf.Ticker(ticker) #, session=session
        yahooHistory = yahooTickerData.history(period="5y", auto_adjust=False, back_adjust=False) #"max"
        yahooHistory['Ticker']=ticker
        yahooHistory = yahooHistory[['Close','Ticker']]
        historyDfs.append(yahooHistory)
    except:
        print(ticker)
        pass

historyDfFull = pd.concat(historyDfs)
historyDfFull.head()

,Close,Ticker
Date,,
2019-05-28 00:00:00-04:00,67.080002,A
2019-05-29 00:00:00-04:00,67.300003,A
2019-05-30 00:00:00-04:00,66.889999,A
2019-05-31 00:00:00-04:00,67.050003,A
2019-06-03 00:00:00-04:00,66.989998,A


In [134]:
# Step 2: Pivot the DataFrame
pivoted_historyDfFull = historyDfFull.pivot(columns='Ticker', values='Close')

# Step 3: Handle missing values if necessary (optional)
pivoted_historyDfFull = pivoted_historyDfFull.fillna(method='ffill').fillna(method='bfill')

display(pivoted_historyDfFull.head())

Ticker,A,ADBE,ADSK,AEP,ALB,ALLE,AMD,AMT,AMZN,AVGO
Date,,,,,,,,,,
2019-05-28 00:00:00-04:00,67.080002,278.250000,161.089996,86.669998,67.430000,98.820000,29.049999,202.550003,91.821503,254.479996
2019-05-29 00:00:00-04:00,67.300003,273.380005,161.520004,85.690002,65.800003,97.790001,28.090000,201.130005,90.959503,254.119995
2019-05-30 00:00:00-04:00,66.889999,274.510010,162.619995,85.300003,65.570000,99.480003,28.030001,204.690002,90.816002,256.619995
2019-05-31 00:00:00-04:00,67.050003,270.899994,160.910004,86.120003,63.299999,97.050003,27.410000,208.770004,88.753502,251.639999
2019-06-03 00:00:00-04:00,66.989998,259.029999,154.820007,87.370003,65.050003,99.599998,27.580000,209.289993,84.634499,253.179993


In [135]:
from scipy.optimize import minimize
returns = pivoted_historyDfFull.pct_change().mean() * 252
cov_matrix = pivoted_historyDfFull.pct_change().cov() * 252
indvStdDevs = pivoted_historyDfFull.pct_change().std() * np.sqrt(252)

In [136]:
returns
cov_matrix
indvStdDevs
indvStdDevsDf = pd.DataFrame(indvStdDevs).rename(columns={0:'Volatility'}).reset_index()
returnsDf = pd.DataFrame(returns).rename(columns={0:'Return'}).reset_index()
returnsDf

,Ticker,Return
0,A,0.204377
1,ADBE,0.178343
2,ADSK,0.135394
3,AEP,0.035474
4,ALB,0.262768
5,ALLE,0.090862
6,AMD,0.486780
7,AMT,0.028376
8,AMZN,0.197279
9,AVGO,0.412769


In [137]:
# Assuming equal weights for simplicity, or you can specify your own weights
num_assets = len(returns)
weights = np.ones(num_assets) / num_assets

# Calculate portfolio return
portfolio_return = np.sum(returns * weights)

# Calculate portfolio standard deviation
portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Calculate Sharpe ratio
risk_free_rate = 0.0  # Assuming risk-free rate is 0 for simplicity
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev

print("Portfolio Return:", portfolio_return)
print("Portfolio Standard Deviation:", portfolio_std_dev)
print("Sharpe Ratio:", sharpe_ratio)

# Step 3: Optimize portfolios for maximum Sharpe ratio
def minimize_negative_sharpe(weights, returns, cov_matrix):
    portfolio_return = np.sum(returns * weights)
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -portfolio_return / portfolio_std_dev

def check_sum(weights):
    return np.sum(weights) - 1

initial_guess = np.array([1 / num_assets] * num_assets)
constraints = ({'type':'eq', 'fun': check_sum})
optimal_sharpe = minimize(minimize_negative_sharpe, initial_guess, args=(returns, cov_matrix), method='SLSQP', bounds=[(0, 1)] * num_assets, constraints=constraints)

# Get the optimal weights
optimal_weights = optimal_sharpe.x

# Calculate the return and standard deviation of the optimal portfolio
optimal_return = np.sum(returns * optimal_weights)
optimal_std_dev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))

print("Optimal Portfolio Weights:", optimal_weights)
print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Standard Deviation:", optimal_std_dev)

# Step 4: Visualize the efficient frontier with your portfolio and the optimal portfolio
# Generate a range of returns for plotting the efficient frontier
target_returns = np.linspace(returns.min(), returns.max(), 100)
frontier_std_dev = []

for target_return in target_returns:
    constraints = ({'type':'eq', 'fun': check_sum}, {'type':'eq', 'fun': lambda weights: np.sum(returns * weights) - target_return})
    result = minimize(lambda weights: np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))), initial_guess, method='SLSQP', bounds=[(0, 1)] * num_assets, constraints=constraints)
    frontier_std_dev.append(result.fun)

# plt.plot(frontier_std_dev, target_returns, label='Efficient Frontier')
# plt.scatter(portfolio_std_dev, portfolio_return, c='red', marker='o', label='Your Portfolio')
# plt.scatter(optimal_std_dev, optimal_return, c='blue', marker='*', s=100, label='Optimal Portfolio')
# plt.xlabel('Volatility (Standard Deviation)')
# plt.ylabel('Return')
# plt.title('Efficient Frontier')
# plt.legend()
# plt.show()

Portfolio Return: 0.20324227801542474
Portfolio Standard Deviation: 0.26301741558886804
Sharpe Ratio: 0.7727331574617858
Optimal Portfolio Weights: [1.25940293e-01 0.00000000e+00 2.37440276e-16 0.00000000e+00
 1.66099773e-16 0.00000000e+00 2.08517641e-01 2.04914211e-17
 0.00000000e+00 6.65542066e-01]
Optimal Portfolio Return: 0.4019565712476886
Optimal Portfolio Standard Deviation: 0.3485237847900516


In [138]:
color_rng =['#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',
            '#997B66','#E8AC65','#FFCB69','#E2D5CD','#9B8E85','#D08C60','#797D62','#D9AE94','#B58463','#F1DCA7','#9B9B7A','#BAA587',]

# Prepare data for visualization
frontier_data = pd.DataFrame({
    'Return': target_returns,
    'Volatility': frontier_std_dev
})

portfolio_data = pd.DataFrame({
    'Return': [portfolio_return, optimal_return],
    'Volatility': [portfolio_std_dev, optimal_std_dev],
    'Type': ['Your Portfolio', 'Optimal Portfolio']
})

optimal_weights_df = pd.DataFrame({
    'Ticker': returns.index,
    'Weight': optimal_weights
})

optimal_weights_df = optimal_weights_df.merge(indvStdDevsDf,how='left',on='Ticker')
optimal_weights_df = optimal_weights_df.merge(returnsDf,how='left',on='Ticker')

# Rename columns
portfolio_data.rename(columns={'Type': 'Entity'}, inplace=True)
optimal_weights_df.rename(columns={'Ticker': 'Entity'}, inplace=True)

# Concatenate portfolio_data and a subset of optimal_weights_df
return_volatility_df = pd.concat([portfolio_data, optimal_weights_df[['Entity', 'Return', 'Volatility']]])


# Altair visualizations
base = alt.Chart(frontier_data).encode(
    x=alt.X('Volatility', axis=alt.Axis(title='Volatility (Standard Deviation)')),
    y=alt.Y('Return', axis=alt.Axis(title='Return', format='%'))
)

# Efficient Frontier
efficient_frontier = base.mark_line(color='#8c8c8c', size=1).encode()

# Individual Portfolio Dots
portfolios = alt.Chart(return_volatility_df).mark_point(size=150, filled=True, opacity=1).encode(
    x='Volatility',
    y='Return',
    color=alt.Color('Entity', scale=alt.Scale(domain=optimal_weights_df['Entity'].values, range=color_rng), legend=alt.Legend(title="Entity"))
)

# Your Portfolio and Optimal Portfolio points
your_optimal_portfolios = alt.Chart(return_volatility_df).mark_point(filled=True, size=150, opacity=1, color='#696969').encode(
    x='Volatility',
    y='Return',
    # color=alt.Color('Entity', scale=alt.Scale(domain=['Your Portfolio', 'Optimal Portfolio'], range=['#FFCB69', '#797D62']), legend=alt.Legend(title="Portfolio Type")),
    shape=alt.Shape('Entity', scale=alt.Scale(domain=['Your Portfolio', 'Optimal Portfolio'], range=['square','triangle']), legend=alt.Legend(title="Portfolio Type"))
)

# Combine the plots
final_chart = (efficient_frontier + your_optimal_portfolios + portfolios).properties(
    width=600,
    height=400,
    title=alt.TitleParams(
        text="Efficient Frontier with Portfolios",
        fontSize=24,
        font='Playfair Display',
        fontWeight='normal',
        color="#696969",
        anchor="start"
    )).configure_axis(
    labelFont="Playfair Display",
    titleFont="Playfair Display",
    labelFontSize=18,
    titleFontSize=14,
    labelColor="#696969",
    titleColor="#696969",
    grid=False  # Remove gridlines
).configure_text(
    font='Playfair Display',
    fontSize=12,
    align='left',
    color="#696969"
).configure_legend(
    titleFont='Playfair Display',
    titleFontSize=18,
    labelFont='Playfair Display',
    labelFontSize=16,
    titleColor="#696969",
    labelColor="#696969"
)

final_chart

alt.LayerChart(...)

In [139]:
# Specify the directory path
directory = "C:/Financial Outputs/Overviews"

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Specify the path and filename for the PNG image
png_file = os.path.join(directory, "modern_portolio_theory_chart.png")

# Save the Altair chart as a PNG image
final_chart.save(png_file, engine="vl-convert")

In [140]:
optimal_weights_df = optimal_weights_df.rename(columns={'Entity':'Ticker'})

# Optimal Portfolio Weights Bar Chart
base_bar = alt.Chart(optimal_weights_df).encode(
    x=alt.X('Weight', axis=alt.Axis(format='%', title=None)),
    y=alt.Y('Ticker', axis=alt.Axis(title=None), sort='-x'),
    color=alt.Color('Ticker', scale=alt.Scale(range=color_rng))
)

bar_chart = base_bar.mark_bar().properties(
    height=300,
    width=600,
    title=alt.TitleParams(
        text="Optimal Portfolio Weights",
        fontSize=24,
        font='Playfair Display',
        fontWeight='normal',
        color="#696969",
        anchor="start"
    )).configure_axis(
    labelFont="Playfair Display",
    titleFont="Playfair Display",
    labelFontSize=18,
    titleFontSize=14,
    labelColor="#696969",
    titleColor="#696969",
    grid=False  # Remove gridlines
).configure_text(
    font='Playfair Display',
    fontSize=12,
    align='left',
    color="#696969"
).configure_legend(
    titleFont='Playfair Display',
    titleFontSize=18,
    labelFont='Playfair Display',
    labelFontSize=16,
    titleColor="#696969",
    labelColor="#696969"
)

bar_chart

alt.Chart(...)

In [141]:
# Specify the directory path
directory = "C:/Financial Outputs/Overviews"

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Specify the path and filename for the PNG image
png_file = os.path.join(directory, "optimal_portolio_weights_chart.png")

# Save the Altair chart as a PNG image
bar_chart.save(png_file, engine="vl-convert")